# Resampling raster data with Python

¿Alguna vez te ha pasado que compras algo en una tienda, solo para dar unos
cuantos pasos más y en otra tienda encontrar el mismo producto a un precio más
bajo? Bueno, eso mismo me ha pasado cuando encuentro una solución a un problema,
para luego encontrar una solución más sencilla y eficiente. En este caso me pasó
al buscar como re-muestrear datos raster con Python.

Con remuestrear datos raster, nos referimos a cambiar la resolución espacial de
una imagen raster, es decir, cambiar el tamaño de los píxeles. Ya sea para hacer
mas _fina/alta_ o más gruesa/baja la resolución de la imagen (_upsampling_ y
_downsampling_ respectivamente).

Esto puede ser útil al integrar datos de diferentes fuentes, es
decir, si tienes dos imágenes raster con diferentes resoluciones, podemos hacer
que ambas tengan la misma resolución espacial (tamaño de píxel). También puede
ser útil para reducir el tamaño de un archivo raster.

Cabe destacar, que existen diferentes métodos para remuestrear una imagen
raster, algunos de los más comunes son:

- **Nearest neighbor**: Este método asigna el valor del píxel más cercano al
  nuevo píxel.
- **Bilinear**: Este método calcula el valor del nuevo píxel a partir de los
    valores de los píxeles vecinos.
- **Cubic**: Este método calcula el valor del nuevo píxel a partir de los
    valores de los píxeles vecinos, utilizando una función cúbica.
- **Cubic spline**: Este método calcula el valor del nuevo píxel a partir de los
    valores de los píxeles vecinos, utilizando una función cúbica spline.
- **Average**: Este método calcula el valor del nuevo píxel a partir del promedio
    de los valores de los píxeles vecinos.

Para profundizar más en los métodos de remuestreo, puedes consultar la
[documentación de
rasterio](https://rasterio.readthedocs.io/en/latest/topics/resampling.html).

Para este ejemplo utilizaremos datos mensuales en formato netCDF de variables
oceanográficas en el Pacífico Norte. Los datos fueron obtenidos de diferentes
fuentes:

| Variable | Resolución espacial |
|----------|----------------------|
| [Temperatura superficial del mar (SST)](https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplMURSST41mday.graph) | 0.01° |
| [Concentración de clorofila-a (CHL)](https://coastwatch.pfeg.noaa.gov/erddap/griddap/nesdisVHNSQchlaMonthly.graph?chlor_a) | 0.0375° |
| [Velocidad del viento a 10m (U10)](https://data.marine.copernicus.eu/product/WIND_GLO_PHY_CLIMATE_L4_MY_012_003/files?subdataset=cmems_obs-wind_glo_phy_my_l4_P1M_202211) | 0.25° |

El objetivo es pasar de la resolución espacial de 0.01° de la SST a la
resolución espacial de 0.25° de la velocidad del viento a 10m.

Para este ejemplo, utilizaremos las librerías `rasterio` y `xarray`.

## Remuestreo de datos usando `rasterio`

### 1. Convertir netCDF a GeoTIFF

Para remuestrear datos raster con `rasterio`, primero necesitamos leer el
archivo netCDF y convertirlo a un archivo GeoTIFF. En este caso vamos a
convertir los 3 archivos netCDF a GeoTIFF.

In [9]:
import xarray as xr

# Read the SST data file and print its contents to verify what are the variables
# and dimensions

# Define the path to the SST data file
sst_file = '../raster_data/2014_01_jplMURSST41.nc'

# Open the dataset and print its contents
sst_ds = xr.open_dataset(sst_file)
print('Metadata of the dataset: \n', sst_ds)

Metadata of the dataset: 
 <xarray.Dataset> Size: 68MB
Dimensions:    (time: 1, latitude: 3201, longitude: 5301)
Coordinates:
  * time       (time) datetime64[ns] 8B 2014-01-16
  * latitude   (latitude) float32 13kB 23.0 23.01 23.02 ... 54.98 54.99 55.0
  * longitude  (longitude) float32 21kB -163.0 -163.0 -163.0 ... -110.0 -110.0
Data variables:
    sst        (time, latitude, longitude) float32 68MB ...
Attributes: (12/40)
    cdm_data_type:              Grid
    Conventions:                CF-1.6, COARDS, ACDD-1.3
    creation_date:              2024-02-03
    creator_email:              erd.data@noaa.gov
    creator_name:               NOAA NMFS SWFSC ERD and NOAA NESDIS CoastWatc...
    creator_type:               institution
    ...                         ...
    summary:                    A monthly mean Sea Surface Temperature (SST) ...
    testOutOfDate:              now-60days
    time_coverage_end:          2014-01-16T00:00:00Z
    time_coverage_start:        2014-01-16T00:

Como podemos observar en los metadata de los archivos netCDF, las dimensiones
del archivo son: `(time: 1, latitude: 3201, longitude: 5301)`, mientras que la
variable que nos interesa es `sst` (temperatura superficial del mar).

Así que seleccionaremos la variable `sst` y asignaremos como dimensiones
espaciales: ` latitude` y `longitude`, para luego convertirlo a un archivo
GeoTIFF.

In [10]:
# Select the SST data array and set the spatial dimensions
sst_da = sst_ds['sst']
sst_da.rio.set_spatial_dims('longitude', 'latitude', inplace=True)
print(sst_da)

crs = 'EPSG:4326'
sst_da.rio.write_crs(crs, inplace=True)

# Create directory for GeoTIFF files
import os
geotiff_dir = '../GeoTIFF_files'
if not os.path.exists(geotiff_dir):
    os.makedirs('../GeoTIFF_files')

# Save the data array as a GeoTIFF file
sst_da.rio.to_raster(f'{geotiff_dir}/2014_01_jplMURSST41.tif')

<xarray.DataArray 'sst' (time: 1, latitude: 3201, longitude: 5301)> Size: 68MB
[16968501 values with dtype=float32]
Coordinates:
  * time       (time) datetime64[ns] 8B 2014-01-16
  * latitude   (latitude) float32 13kB 23.0 23.01 23.02 ... 54.98 54.99 55.0
  * longitude  (longitude) float32 21kB -163.0 -163.0 -163.0 ... -110.0 -110.0
Attributes:
    colorBarMaximum:  32.0
    colorBarMinimum:  0.0
    ioos_category:    Temperature
    long_name:        Sea Surface Temperature Monthly Mean
    standard_name:    sea_surface_foundation_temperature
    units:            degree_C


El archivo de la velocidad del viendo, que es nuestra resolución objetivo
(0.25°) también debemos de covertirlo a un archivo GeoTIFF.

In [11]:
# Read the wind data file and print its contents to verify what are the variables

# Define the path to the wind data file
wind_file = '../raster_data/2014_01_cmems_obs-wind_l4.nc'
wind_ds = xr.open_dataset(wind_file)
print('Metadata of the dataset: \n', wind_ds)

Metadata of the dataset: 
 <xarray.Dataset> Size: 3MB
Dimensions:                 (time: 1, lat: 128, lon: 212)
Coordinates:
  * time                    (time) datetime64[ns] 8B 2014-01-16T12:00:00
  * lat                     (lat) float32 512B 23.12 23.38 23.62 ... 54.62 54.88
  * lon                     (lon) float32 848B -162.9 -162.6 ... -110.4 -110.1
Data variables: (12/13)
    eastward_wind           (time, lat, lon) float64 217kB ...
    eastward_wind_bias      (time, lat, lon) float64 217kB ...
    eastward_wind_sdd       (time, lat, lon) float64 217kB ...
    northward_wind          (time, lat, lon) float64 217kB ...
    northward_wind_bias     (time, lat, lon) float64 217kB ...
    northward_wind_sdd      (time, lat, lon) float64 217kB ...
    ...                      ...
    eastward_stress_bias    (time, lat, lon) float64 217kB ...
    eastward_stress_sdd     (time, lat, lon) float64 217kB ...
    northward_stress        (time, lat, lon) float64 217kB ...
    northward_stre

In [12]:
# Select the eastward wind data array and set the spatial dimensions
wind_da = wind_ds['eastward_wind']
wind_da.rio.set_spatial_dims('lon', 'lat', inplace=True)
print(wind_da)

wind_da.rio.write_crs(crs, inplace=True)

# Save the data array as a GeoTIFF file
wind_da.rio.to_raster(f'{geotiff_dir}/2014_01_cmems_obs-wind_l4.tif')

<xarray.DataArray 'eastward_wind' (time: 1, lat: 128, lon: 212)> Size: 217kB
[27136 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 8B 2014-01-16T12:00:00
  * lat      (lat) float32 512B 23.12 23.38 23.62 23.88 ... 54.38 54.62 54.88
  * lon      (lon) float32 848B -162.9 -162.6 -162.4 ... -110.6 -110.4 -110.1
Attributes:
    units:          m s-1
    long_name:      stress-equivalent wind eastward component at 10 m
    standard_name:  eastward_wind
    valid_min:      -5000
    valid_max:      5000


Como podemos observar, las dimensiones de las matrices de datos son diferentes:

1. Temperatura superficial del mar (0.01°): `(time: 1, latitude: 3201, longitude: 5301)`
2. Velocidad del viento a 10m (0.25°): `(time: 1, latitude: 128, longitude: 212)`

Los datos de la SST tienen una mayor cantidad de píxeles que los datos de la
velocidad del viento.

### 2. Remuestreo de datos con `rasterio`

Una vez que tenemos los archivos GeoTIFF, podemos remuestrear los datos de la
SST a la resolución de la velocidad del viento. Para ello, utilizaremos la
función sugerida en la [documentación de
`rasterio`](https://pygis.io/docs/e_raster_resample.html#example-of-co-registering-rasters-with-rasterio).
para remuestrear/reprojectar datos.


In [13]:
from rasterio.warp import reproject, Resampling, calculate_default_transform
import rasterio

def reproj_match(infile, match, outfile):
    """Reproject a file to match the shape and projection of existing raster. 
    
    Parameters
    ----------
    infile : (string) path to input file to reproject
    match : (string) path to raster with desired shape and projection 
    outfile : (string) path to output file tif
    """
    # open input
    with rasterio.open(infile) as src:
        src_transform = src.transform
        
        # open input to match
        with rasterio.open(match) as match:
            dst_crs = match.crs
            
            # calculate the output transform matrix
            dst_transform, dst_width, dst_height = calculate_default_transform(
                src.crs,     # input CRS
                dst_crs,     # output CRS
                match.width,   # input width
                match.height,  # input height 
                *match.bounds,  # unpacks input outer boundaries (left, bottom, right, top)
            )

        # set properties for output
        dst_kwargs = src.meta.copy()
        dst_kwargs.update({"crs": dst_crs,
                           "transform": dst_transform,
                           "width": dst_width,
                           "height": dst_height,
                           "nodata": 0})
        print("Coregistered to shape:", dst_height,dst_width,'\n Affine',dst_transform)
        # open output
        with rasterio.open(outfile, "w", **dst_kwargs) as dst:
            # iterate through bands and write using reproject function
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=dst_transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)


Y ahora sí, utilizando la función anterior vamos a remuestrear los datos de la
SST (`input_file`) a la resolución de la velocidad del viento
(`match_file`) y guardaremos el resultado en un nuevo archivo GeoTIFF
(`output_file`).

In [14]:
sst_tif = f'{geotiff_dir}/2014_01_jplMURSST41.tif'
wind_tif = f'{geotiff_dir}/2014_01_cmems_obs-wind_l4.tif'

